In [1]:
from atomistics.workflows.elastic.workflow import (
    analyse_structures_helper,
    generate_structures_helper
)
from atomistics.calculators import (
    evaluate_with_lammps, get_potential_by_name
)
from h5io_browser.pointer import get_hierarchical_dict
from pyiron_base.storage.hdfio import ProjectHDFio, _extract_module_class_name, _import_class

In [2]:
from ase.build import bulk
from pyiron_base import Project

In [3]:
def convert_job_dict_to_job(project, job_name, job_type):
    module_path, class_name = _extract_module_class_name(job_type)
    class_object = _import_class(module_path, class_name)
    project_hdf = ProjectHDFio(project=project, file_name=job_name)
    return class_object(project_hdf, job_name)

In [4]:
def calc_with_lammps(structure, potential_name, calc_mode="static"):
    df_pot_selected = get_potential_by_name(
        potential_name=potential_name,
    )
    if calc_mode == "minimize":
        return {"structure": evaluate_with_lammps(
            task_dict={"optimize_positions_and_volume": structure},
            potential_dataframe=df_pot_selected,
        )['structure_with_optimized_positions_and_volume']}
    elif calc_mode == "static":
        return evaluate_with_lammps(
            task_dict={"calc_energy": structure},
            potential_dataframe=df_pot_selected,
        )
    else:
        raise ValueError()

In [5]:
def copy_job(ref_job):
    job_ref_dict = job_ref.to_dict()
    job_copy = convert_job_dict_to_job(project=ref_job.project, job_name=ref_job.job_name, job_type=job_ref_dict["TYPE"])
    job_copy.from_dict(job_dict=get_hierarchical_dict(job_ref_dict))
    for k, v in job_ref.input.to_builtin().items():
        job_copy.input[k] = v
    return job_copy

In [6]:
def calc_energy_volume_curve(ref_job, eps_range=0.005, num_of_point=5, fit_order=2):
    sym_dict, structure_dict = generate_structures_helper(
        structure=ref_job.input.structure,
        eps_range=eps_range,
        num_of_point=num_of_point,
    )
    energy_dict = {}
    for strain, struct in structure_dict.items():
        job_strain = copy_job(ref_job=ref_job)
        job_strain.input.structure = struct
        job_strain.run()
        energy_dict[strain] = job_strain.output.result["energy"]
    return analyse_structures_helper(
        output_dict={"energy": energy_dict},
        sym_dict=sym_dict,
        fit_order=2
    )

In [7]:
pr = Project("test")

In [8]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/13 [00:00<?, ?it/s]

In [9]:
structure = bulk("Al", cubic=True)

In [10]:
job_mini = pr.wrap_python_function(calc_with_lammps)
job_mini.input.structure = structure
job_mini.input.potential_name = '1999--Mishin-Y--Al--LAMMPS--ipr1'
job_mini.input.calc_mode="minimize"
job_mini.run()

The job calc_with_lammps9584dea7d794ef2c901fe7d264464cee was saved and received the ID: calc_with_lammps9584dea7d794ef2c901fe7d264464cee


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


In [11]:
job_ref = pr.wrap_python_function(calc_with_lammps)
job_ref.input.structure = job_mini.output.result['structure']
job_ref.input.potential_name = '1999--Mishin-Y--Al--LAMMPS--ipr1'

In [12]:
job_murn = pr.wrap_python_function(calc_energy_volume_curve)
job_murn.input.ref_job = job_ref
job_murn.input.eps_range = 0.005
job_murn.input.num_of_point = 5
job_murn.input.fit_order = 2
job_murn.run()

/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


The job calc_energy_volume_curve0eb680fcb55f24070c413ae63fde62bc was saved and received the ID: calc_energy_volume_curve0eb680fcb55f24070c413ae63fde62bc
The job calc_with_lammpsff5e993f252fa4d56a980f5ab41f854f was saved and received the ID: calc_with_lammpsff5e993f252fa4d56a980f5ab41f854f
The job calc_with_lammps54f632938ca3f4a5cf132eb6e2a3de2c was saved and received the ID: calc_with_lammps54f632938ca3f4a5cf132eb6e2a3de2c


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/User

The job calc_with_lammpsbfbe1f35d9f685196c72703f34c3a5e6 was saved and received the ID: calc_with_lammpsbfbe1f35d9f685196c72703f34c3a5e6
The job calc_with_lammps3c75feadebdf5a32700c6c409fb1b9a5 was saved and received the ID: calc_with_lammps3c75feadebdf5a32700c6c409fb1b9a5
The job calc_with_lammpsea6a2dc88441d3fb0f6975128efda4e5 was saved and received the ID: calc_with_lammpsea6a2dc88441d3fb0f6975128efda4e5


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/User

...

/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/User

The job calc_with_lammpsc7af01b1d9a08c123a9f020730186b28 was saved and received the ID: calc_with_lammpsc7af01b1d9a08c123a9f020730186b28
The job calc_with_lammpsec0866e1e6a9aaf57933420f57fb8127 was saved and received the ID: calc_with_lammpsec0866e1e6a9aaf57933420f57fb8127
The job calc_with_lammps2b96e27468594988044dd9be8c5e5ce7 was saved and received the ID: calc_with_lammps2b96e27468594988044dd9be8c5e5ce7


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


The job calc_with_lammps0ba605d54d4cbfea54f26b5eed29aebf was saved and received the ID: calc_with_lammps0ba605d54d4cbfea54f26b5eed29aebf
The job calc_with_lammps2be97efc2aa7bc2513d2a4d71282788b was saved and received the ID: calc_with_lammps2be97efc2aa7bc2513d2a4d71282788b


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


In [13]:
job_murn.output["result"]

({'SGN': 225,
  'v0': 66.43035441556098,
  'LC': 'CI',
  'Lag_strain_list': ['01', '08', '23'],
  'epss': array([-0.005 , -0.0025,  0.    ,  0.0025,  0.005 ]),
  'strain_energy': [[(-0.005, -13.436320248980278),
    (-0.0025, -13.439079680886989),
    (0.0, -13.439999952735112),
    (0.0024999999999999996, -13.439084974614394),
    (0.005, -13.436364320399795)],
   [(-0.005, -13.43817471490433),
    (-0.0025, -13.439544638502628),
    (0.0, -13.439999952735112),
    (0.0024999999999999996, -13.43954822781134),
    (0.005, -13.438204192615181)],
   [(-0.005, -13.436741954502294),
    (-0.0025, -13.439195465714546),
    (0.0, -13.439999952735112),
    (0.0024999999999999996, -13.439213491269703),
    (0.005, -13.436885713447483)]],
  'e0': -13.439999952735112,
  'A2': array([2.20130388, 1.08985578, 1.91883479])},
 {'elastic_matrix': array([[114.10311701,  60.51102935,  60.51102935,   0.        ,
            0.        ,   0.        ],
         [ 60.51102935, 114.10311701,  60.51102935,   